# Component 1+2 :: Acquire Reddit Data & Cached-Analyze via API

Goal of notebook ::
- Grab data from Reddit
- Load in a cache dictionary
- Check each row's ID's to see if we've pinged the API for analysis already
- If `redditThing.id not in cache.keys()` run the body text through the API query and save the results

## Setup
- Mount the Google Drive
- Setup the Google Cloud Prerequisites & let the kernel restart happen
- Setup the Google Cloud Credentials
- Setup the PRAW package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## install this for first latest cloud language API. Run this before the next one. Next 'pip install' will make you restart runtime, that's fine
## for some reason the sample function doesn't run without this.
!pip install --upgrade -q google-cloud-language

     |████████████████████████████████| 153kB 11.3MB/s 
     |████████████████████████████████| 51kB 3.8MB/s 
     |████████████████████████████████| 522kB 10.6MB/s 
     |████████████████████████████████| 645kB 34.5MB/s 


In [ ]:
## instaling the V1 version of the API to get the entity-sentiment demo working
## it will ask to restart runtime. Restart it and run this again. Then run the next cell.
!pip install gapic-google-cloud-language-v1

     |████████████████████████████████| 51kB 3.4MB/s 
     |████████████████████████████████| 81kB 4.3MB/s 
     |████████████████████████████████| 153kB 17.1MB/s 
     |████████████████████████████████| 163kB 16.0MB/s 
  Created wheel for gapic-google-cloud-language-v1: filename=gapic_google_cloud_language_v1-0.15.3-cp37-none-any.whl size=16502 sha256=4623031df4519788efad52e9fa54277c106aaeab8f471677f584c0b6501b93e5
  Stored in directory: /root/.cache/pip/wheels/b6/27/56/2f272e4d5596569f2612765054e20e215377f32e330e60fda3
  Created wheel for oauth2client: filename=oauth2client-3.0.0-cp37-none-any.whl size=106383 sha256=b5f7e7f993a55f27ea765d7ccd762bfb202a77102001f126c8c38d791548cd82
  Stored in directory: /root/.cache/pip/wheels/48/f7/87/b932f09c6335dbcf45d916937105a372ab14f353a9ca431d7d
  Created wheel for proto-google-cloud-language-v1: filename=proto_google_cloud_language_v1-0.15.3-cp37-none-any.whl size=19048 sha256=944f1a31502c75157481b378797b755f4acd749beb63ba45eabcd008a8ce75e6
  

In [ ]:
! pip install praw

     |████████████████████████████████| 163kB 12.4MB/s 
     |████████████████████████████████| 61kB 4.0MB/s 


In [ ]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/drive/Shareddrives/LHS712 Project [NLP]/key/LHS712-c318b48fbe5f.json"
!gcloud auth activate-service-account --key-file '/content/drive/Shareddrives/LHS712 Project [NLP]/key/LHS712-c318b48fbe5f.json'

Activated service account credentials for: [nortegag@lhs712.iam.gserviceaccount.com]


In [ ]:
# set up environment, app credentials are in the Drive, use them
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/drive/Shareddrives/LHS712 Project [NLP]/key/LHS712-c318b48fbe5f.json'

## Getting Live Reddit Data

To-Do's:
- Add support for gather-method differentiation, `getter(... how=["new","hot"], what=["posts","comments","replies])`
- Add support for gathering responses to comments, accessible as `commentObject.replies.list()` and probably with similar handling of See-More via exception or, at cost of an API call per See-More, evaluation

In [ ]:
with open("/content/drive/Shareddrives/LHS712 Project [NLP]/key/RedditKeys.txt") as file:
    Client = file.readline().split(',')[1].strip()
    Secret = file.readline().split(',')[1].strip()
    Agent = file.readline().split(',')[1].strip()

In [ ]:
import praw
reddit = praw.Reddit(client_id=Client,
                     client_secret=Secret,
                     user_agent=Agent,
                     check_for_async=False)

In [ ]:
def getter(connector, subreddits=[], n_posts=10, v=1):
    import pandas as pd
    df = pd.DataFrame(columns=['subreddit','id','permalink','type','title','text'])
    #### Connect to each subreddit and get N posts from Hot or New
    #### TO DO -- parameterize for hot // new // both ?
    for subreddit_name in subreddits:
        print(f"Active :: r/{subreddit_name}")
        subreddit_connection = reddit.subreddit(subreddit_name)
        sample = subreddit_connection.hot(limit=n_posts)
        
        for post in sample:
            df = df.append({"subreddit":post.subreddit, "id":post.id,
                            "permalink":post.permalink, "type":"post",
                            "title":post.title, "text":post.selftext}, ignore_index=True)

            for comment in post.comments.list():        
                #### Try-Except handling incase of a MoreComment object ("See More")
                try:
                    df = df.append({"subreddit":comment.subreddit, "id":comment.id,
                            "permalink":comment.permalink, "type":"comment",
                            "title":None, "text":comment.body}, ignore_index=True)
                except:
                    pass
    return(df)

In [ ]:
#### JUST get original posts
def fullPostGetter(connector, subreddits=[], n_posts=10, v=1):
    import pandas as pd
    df = pd.DataFrame(columns=['subreddit','id','permalink','type','title','text'])
    #### Connect to each subreddit and get N posts from Hot or New
    #### TO DO -- parameterize for hot // new // both ?
    for subreddit_name in subreddits:
        print(f"Active :: r/{subreddit_name}")
        subreddit_connection = reddit.subreddit(subreddit_name)
        sample = subreddit_connection.hot(limit=n_posts)
        
        for post in sample:
            df = df.append({"subreddit":post.subreddit, "id":post.id,
                            "permalink":post.permalink, "type":"post",
                            "title":post.title, "text":post.selftext}, ignore_index=True)
    return(df)

In [ ]:
#reddit_data = getter(connector=reddit, subreddits=['meth'], n_posts=50, v=0)
#reddit_data = fullPostGetter(connector=reddit, subreddits=['heroin','ketamine','mdma','opiates','cocaine','dmt','lsd'], n_posts=250, v=0)
#reddit_data = fullPostGetter(connector=reddit, subreddits=['anxiety','depression','psychonaut','addiction'], n_posts=250, v=0)
#reddit_data = fullPostGetter(connector=reddit, subreddits=['trees','leaves','petioles','addiction',
#                                                           'drugs','stims','microdosing','redditorsinrecovery'], n_posts=250, v=0)
#reddit_data = getter(connector=reddit, subreddits=['bipolar'], n_posts=50, v=0)
#reddit_data = getter(connector=reddit, subreddits=['eatingdisorders'], n_posts=50, v=0)
#reddit_data = fullPostGetter(connector=reddit, subreddits=['eatingdisorders','drugs','addiction'], n_posts=100, v=0)

# 'depression','eatingdisorders'
# reddit_data = fullPostGetter(connector=reddit, subreddits=['addiction','depression'], n_posts=500, v=0)
reddit_data = fullPostGetter(connector=reddit, subreddits=['leaves','petioles'], n_posts=500, v=0)

print(f"{len(reddit_data)} total posts/comments gathered")

Active :: r/leaves
Active :: r/petioles
1000 total posts/comments gathered


In [ ]:
reddit_data[['subreddit','id','type','title','text','permalink']].sample(5)

,subreddit,id,type,title,text,permalink
768,Petioles,mkmh5p,post,After my recent T-break/cutting back in the pa...,I really realized how much I was smoking and h...,/r/Petioles/comments/mkmh5p/after_my_recent_tb...
290,leaves,mtow8f,post,personality already coming back,"gonna try to make it brief. 18F, been smoking ...",/r/leaves/comments/mtow8f/personality_already_...
754,Petioles,mlv3mf,post,Why is this such a pain,I told myself I am going to not have any T ton...,/r/Petioles/comments/mlv3mf/why_is_this_such_a...
921,Petioles,mdflxq,post,Seeing all the mentally vapid posts on r/highi...,Title,/r/Petioles/comments/mdflxq/seeing_all_the_men...
104,leaves,mv1ryd,post,112 days,Happy 420 y’all! I gotta admit it feels so goo...,/r/leaves/comments/mv1ryd/112_days/


## Sentiment Analysis

Pipeline Thought Draft

* Grab Data --> Into Dataframe
* Create/Import a dictionary
* For row in DataFrame, lookup if ID is in dictionary.keys()
* * If no, THEN do the API call
* * Take the analysis response object and store it into <br/>`data[id]={`<br/>`"response":responseObject,`<br/>`"type":post/comment/etc,`<br/>`"text":"i got regression depression",`<br/>`"subreddit":"r/something"`<br/>`}` 
* Dump the dictionary to a JSON file


In [ ]:
import json

try:
    with open('/content/drive/Shareddrives/LHS712 Project [NLP]/data/data_hoard.json', 'r') as infile:
        data_hoard = json.load(infile)
        print("data_hoard.json found and loaded")
except:
    print("data_hoard.json NOT FOUND, initializing {}")
    data_hoard = {}

data_hoard.json found and loaded


In [ ]:
from google.cloud import language_v1

def sample_analyze_entity_sentiment(text_content, v=0):
    # Analyzing Entity Sentiment in a String
    # Args: text_content The text content to analyze

    client = language_v1.LanguageServiceClient()

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    document = {"content": text_content, "type_": type_, "language": "en"}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_entity_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    # Loop through entitites returned from the API
    for entity in response.entities:
        if(v):print(f"\tEntity Name: {entity.name}")
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        if(v):print(f"Entity Type: {language_v1.Entity.Type(entity.type_).name}")
        # Get the salience score associated with the entity in the [0, 1.0] range
        if(v):print(u"Salience score: {}".format(entity.salience))
        # Get the aggregate sentiment expressed for this entity in the provided document.
        sentiment = entity.sentiment
        if(v):print(u"Entity sentiment score: {}".format(sentiment.score))
        if(v):print(u"Entity sentiment magnitude: {}".format(sentiment.magnitude))
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        for metadata_name, metadata_value in entity.metadata.items():
            if(v):print(u"{} = {}".format(metadata_name, metadata_value))

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            if(v):print(u"Mention text: {}".format(mention.text.content))
            # Get the mention type, e.g. PROPER for proper noun
            if(v):print(f"Mention type: {language_v1.EntityMention.Type(mention.type_).name}")

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    if(v):print(u"Language of the text: {}".format(response.language))

    return(response)

In [ ]:
#### Google's format won't convert to json or dict or serialize as pickle or parquet
#### Convert as ._pb then convert to dict/json
# https://stackoverflow.com/questions/19734617/protobuf-to-json-in-python
from google.protobuf.json_format import MessageToDict, MessageToJson 

for rowNum in range(reddit_data.shape[0]):
    targetRow = reddit_data.iloc[rowNum]

    if(targetRow.id not in data_hoard.keys()):
        data_hoard[targetRow.id] = {
            "subreddit":targetRow.subreddit.display_name,
            "permalink":targetRow.permalink,
            "type":targetRow.type,
            "text":targetRow.text,
            "response":MessageToDict(sample_analyze_entity_sentiment(text_content=targetRow.text, v=0)._pb)
        }
        print(f"id={targetRow.id} added...")
    else:
        print(f"id={targetRow.id} is duplicate, passing...")

id=mugsqd added...
id=gfd094 is duplicate, passing...
id=mvgl7x added...
id=mvki3c added...
id=mv696p added...
id=mvedp7 added...
id=mvjyvj added...
id=mvbl57 added...
id=mvip3h added...
id=mv6nz0 added...
id=mvgeyz added...
id=mvhxyt added...
id=mvnwe0 added...
id=mviejk added...
id=mv3fgf added...
id=mvooo1 added...
id=muyr0q added...
id=mvpogc added...
id=mvna9w added...
id=mvbjqr added...
id=mvok9d added...
id=mv4azx added...
id=mv73rj added...
id=mvmp70 added...
id=mvmj2a added...
id=mvljn3 added...
id=mvlh9z added...
id=mvfjyo added...
id=mvd3lr added...
id=muor0v added...
id=mvnp54 added...
id=mvkog6 added...
id=mvng8k added...
id=mvi7dx added...
id=mvhsj9 added...
id=mvjsa2 added...
id=mvh9ae added...
id=mvly52 added...
id=mv94lf added...
id=mvlx9p added...
id=mv0nex added...
id=mvqdjz added...
id=mvq2r4 added...
id=mvl7kx added...
id=mvq0op added...
id=mvl5b6 added...
id=mvl2wr added...
id=mvpszp added...
id=mv986d added...
id=mvhmxs added...
id=mvfe5z added...
id=mv93jd added

In [ ]:
len(data_hoard.keys())

24174

In [ ]:
import json
with open("/content/drive/Shareddrives/LHS712 Project [NLP]/data/data_hoard.json", "w") as outfile:
    json.dump(data_hoard, outfile)

## Miscellaneous

In [ ]:
#### Keep :: McCoy's example for "oh wow it really just neglected sentiment for some of these"
single = sample_analyze_entity_sentiment(data_hoard['msaugy']['text'])

In [ ]:
single.response

AttributeError: ignored

In [ ]:
[i.name for i in single.respose.]

AttributeError: ignored

In [ ]:
list(data_hoard.keys())[:4]

['msaugy', 'guriwq1', 'gurghwo', 'gurspvm']

In [ ]:
[i['name'] for i in data_hoard['msaugy']['response']['entities']]

['flower',
 'challenges',
 'risk',
 'everyone',
 'temptation',
 'dabs',
 'idea',
 'challenge',
 'stores',
 'challenge',
 'dispensaries',
 'dream',
 'scene',
 'forms',
 'vapes',
 'manner',
 'extracts',
 'delivery services',
 'things',
 'offer',
 'risk',
 'coping skills',
 'Delta 8',
 'use',
 'price',
 'high',
 'reports',
 'process',
 'cannabis companies',
 'skills',
 'Leaves',
 'problems',
 'something',
 'New York City',
 'suggestion',
 'Delta 9',
 'urge',
 'envy',
 'thing',
 'focus',
 'bedtime',
 'anything',
 'something',
 'sobriety']

In [ ]:
data_hoard['msaugy']['response']

{'entities': [{'mentions': [{'sentiment': {'magnitude': 0.2, 'score': 0.2},
     'text': {'beginOffset': 177, 'content': 'flower'},
     'type': 'COMMON'}],
   'name': 'flower',
   'salience': 0.10788207,
   'sentiment': {'magnitude': 0.3, 'score': 0.1},
   'type': 'OTHER'},
  {'mentions': [{'sentiment': {},
     'text': {'beginOffset': 113, 'content': 'challenges'},
     'type': 'COMMON'}],
   'name': 'challenges',
   'salience': 0.0916746,
   'sentiment': {},
   'type': 'OTHER'},
  {'mentions': [{'sentiment': {},
     'text': {'beginOffset': 974, 'content': 'risk'},
     'type': 'COMMON'}],
   'name': 'risk',
   'salience': 0.077089414,
   'sentiment': {},
   'type': 'OTHER'},
  {'mentions': [{'sentiment': {},
     'text': {'beginOffset': 65, 'content': 'everyone'},
     'type': 'COMMON'}],
   'name': 'everyone',
   'salience': 0.07368546,
   'sentiment': {},
   'type': 'PERSON'},
  {'mentions': [{'sentiment': {},
     'text': {'beginOffset': 1355, 'content': 'temptation'},
     'typ

In [ ]:
data_hoard['msaugy']['response']['entities'][0]

{'mentions': [{'sentiment': {'magnitude': 0.2, 'score': 0.2},
   'text': {'beginOffset': 177, 'content': 'flower'},
   'type': 'COMMON'}],
 'name': 'flower',
 'salience': 0.10788207,
 'sentiment': {'magnitude': 0.3, 'score': 0.1},
 'type': 'OTHER'}

## Traversing the response object

<b>The version for operating directly on response object</b>

In [ ]:
document = data_hoard['msaugy']['text']
doc_response = sample_analyze_entity_sentiment(document, v=0)

for entity in doc_response.entities:
    print(f"\tEntity Name: {entity.name}")
    print(f"Entity Type: {language_v1.Entity.Type(entity.type_).name}")
    print(f"Salience score: {round(entity.salience,6)}") # [0.0 to 1.0]

    # Get the aggregate sentiment expressed for this entity in the provided document.
    sentiment = entity.sentiment
    print(f"Entity sentiment score: {round(sentiment.score,6)}")
    print(f"Entity sentiment magnitude: {round(sentiment.magnitude,6)}")

    # Loop over the metadata associated with entity. For many known entities,
    # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
    #for metadata_name, metadata_value in entity.metadata.items():
        #print(f"\033[91m{metadata_name} = {metadata_value}\x1b[0m")

    # Loop over the mentions of this entity in the input document.
    # The API currently supports proper noun mentions.
    for mention in entity.mentions:
        print(f"Mention text: {mention.text.content}")
        # Get the mention type, e.g. PROPER for proper noun
        print(f"Mention type: {language_v1.EntityMention.Type(mention.type_).name}")
        print(f"Mention Sentiment (Score, Magnitude) :: ({mention.sentiment.score},{mention.sentiment.magnitude})")
    print()

	Entity Name: flower
Entity Type: OTHER
Salience score: 0.107882
Entity sentiment score: 0.1
Entity sentiment magnitude: 0.3
Mention text: flower
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0.20000000298023224,0.20000000298023224)

	Entity Name: challenges
Entity Type: OTHER
Salience score: 0.091675
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Mention text: challenges
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0.0,0.0)

	Entity Name: risk
Entity Type: OTHER
Salience score: 0.077089
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Mention text: risk
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0.0,0.0)

	Entity Name: everyone
Entity Type: PERSON
Salience score: 0.073685
Entity sentiment score: 0.0
Entity sentiment magnitude: 0.0
Mention text: everyone
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0.0,0.0)

	Entity Name: temptation
Entity Type: OTHER
Salience score: 0.071146
Entity sentiment s

<b>The version for operating on the dictionary object</b>
... basically, just use keys instead of attributes

In [ ]:
doc_response = collection_dict['msaugy']['response']
for entity in doc_response['entities']:
    print(f"\tEntity Name: {entity['name']}")
    print(f"Entity Type: {entity['type']}")
    print(f"Salience score: {round(entity['salience'],6)}") # [0.0 to 1.0]

    sentiment = entity['sentiment']
    
    try:
        sent_score = sentiment['score']
    except:
        sent_score = 0
    
    try:
        sent_mag = sentiment['magnitude']
    except:
        sent_mag = 0

    print(f"Entity sentiment score: {round(sent_score,6)}")
    print(f"Entity sentiment magnitude: {round(sent_mag,6)}")

    #### Not sure we'll have any need for this? Not really seeing examples of it either?
    #for metadata_name, metadata_value in entity.metadata.items():
    #    print(f"{metadata_name} = {metadata_value}")

    # Loop over the mentions of this entity in the input document.
    # The API currently supports proper noun mentions.
    for mention in entity['mentions']:
        try:
            mention_mag = mention['sentiment']['magnitude']
        except:
            mention_mag = 0
        try:
            mention_score = mention['sentiment']['score']
        except:
            mention_score = 0
        print(f"Mention text: {mention['text']['content']}")
        # Get the mention type, e.g. PROPER for proper noun
        print(f"Mention type: {mention['type']}")
        print(f"Mention Sentiment (Score, Magnitude) :: ({mention_score},{mention_mag})")

	Entity Name: flower
Entity Type: OTHER
Salience score: 0.107882
Entity sentiment score: 0.1
Entity sentiment magnitude: 0.3
Mention text: flower
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0.2,0.2)
	Entity Name: challenges
Entity Type: OTHER
Salience score: 0.091675
Entity sentiment score: 0
Entity sentiment magnitude: 0
Mention text: challenges
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0,0)
	Entity Name: risk
Entity Type: OTHER
Salience score: 0.077089
Entity sentiment score: 0
Entity sentiment magnitude: 0
Mention text: risk
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0,0)
	Entity Name: everyone
Entity Type: PERSON
Salience score: 0.073685
Entity sentiment score: 0
Entity sentiment magnitude: 0
Mention text: everyone
Mention type: COMMON
Mention Sentiment (Score, Magnitude) :: (0,0)
	Entity Name: temptation
Entity Type: OTHER
Salience score: 0.071146
Entity sentiment score: 0
Entity sentiment magnitude: 0
Mention text: temptati

In [ ]:
collection_dict['msaugy']['response']['entities'][1]

{'mentions': [{'sentiment': {},
   'text': {'beginOffset': 113, 'content': 'challenges'},
   'type': 'COMMON'}],
 'name': 'challenges',
 'salience': 0.0916746,
 'sentiment': {},
 'type': 'OTHER'}

## Pipeline End :: Aggregate & Analyze Results

In [ ]:
#### TO-DO